In [224]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time 
import pandas as pd
from bs4 import BeautifulSoup

In [225]:
import configparser

In [226]:
user_id = "hctmkr"
user_pass = "hcinc3600"

In [227]:
#options = webdriver.ChromeOptions()
driver = webdriver.Chrome('./chromedriver')
driver.set_window_size(1920, 1080)

In [228]:
#driver.set_window_size(1400, 1200)
driver.get("https://www.kaida.co.kr/uat/uia/egovLoginUsr.do")

In [229]:
driver.find_element_by_css_selector('#userId').send_keys(user_id)
driver.find_element_by_css_selector('#userPass').send_keys(user_pass)
driver.execute_script('actionLogin()')

In [230]:
# 2. 사용자 설정통계 페이지 이동

In [231]:
driver.get("https://www.kaida.co.kr/ko/statistics/custom2.do")

In [232]:
# 3. 브랜드, 지역, 구매유형, 연령 데이터 수집

In [233]:
# 구매유형
selector = '#pageBody > article.body_wrap > div > div.select_filter > table > tbody > tr > td:nth-child(8) > div > label'
sales_types = driver.find_elements_by_css_selector(selector)
sales_types = {sales_type.text: sales_type.get_attribute("for") for sales_type in sales_types}
print(len(sales_types), sales_types)

4 {'개인': 'PurchaseType3', '개인-남자': 'PurchaseType1', '개인-여자': 'PurchaseType2', '법인': 'PurchaseType0'}


In [234]:
# 연령
ages = driver.find_elements_by_css_selector(".td_Age > .item_wrap > label")
ages = {age.text: age.get_attribute("for") for age in ages}
print(len(ages), ages)

7 {'~19': 'Age1', '20~29': 'Age2', '30~39': 'Age3', '40~49': 'Age4', '50~59': 'Age5', '60~69': 'Age6', '70~': 'Age7'}


In [235]:
def clear_menu(driver):
    # 나이 체크박스 초기화
    driver.execute_script("$('#td_Age > .item_wrap > label > input').prop('checked', false);")
    # tag 초기화
    driver.execute_script("$('.sel_item button').click();");

In [236]:
def select_menu(driver, sales_type_id, age_id, year):
    
    # 메뉴 초기화
    clear_menu(driver)
    time.sleep(0.1)
    
    # 구매유형 클릭
    driver.find_element_by_css_selector(f'input#{sales_type_id}').click()
    time.sleep(0.1)
    
    # 연령대 클릭
    try: 
        driver.find_element_by_css_selector(f'input#{age_id}').click()
        time.sleep(0.2)
    except:
        pass

In [237]:
def make_table_df(driver, sales_type, age, year):
    
    # 컬럼 데이터 수집
    columns = driver.find_elements_by_css_selector("table.re_table > thead > tr > th")
    columns = [column.text for column in columns] + [ "sales_type", "age", "year"] 

    # 데이터 프레임 만들기
    df = pd.DataFrame(columns=columns)

    # 데이터 프레임 채우기
    rows = driver.find_elements_by_css_selector("table.re_table > tbody > tr")
    for row in rows:
        cells = row.find_elements_by_css_selector('td,th')
        datas = [cell.text for cell in cells] + [sales_type, age, year]     
        df.loc[len(df)] = datas
    
    return df

In [204]:
list(sales_types.items())[3:]

[('법인', 'PurchaseType0')]

In [207]:
dfs = []

for year in range(2016, 2022): #2016, 2022        
    years = Select(driver.find_element_by_xpath('//*[@id="searchYear"]'))
    years.select_by_value(f'{year}')
    
    driver.find_element_by_css_selector('#searchDivision > option:nth-child(2)').click()

    for sales_type, sales_type_id in list(sales_types.items())[3:]:
        for age, age_id in list(ages.items()):

                    # 메뉴 선택
            try:
                select_menu(driver, sales_type_id, age_id, year)
            except:
                pass

                    # 검색버튼 클릭
            driver.find_element_by_css_selector('.searchBtn').click()
            time.sleep(0.2)

                    # 데이터 수집해서 데이터 프레임으로 만들기
            try: 
                df = make_table_df(driver, sales_type, age, year)
                dfs.append(df)
            except:
                pass
                
# 수집된 데이터 프레임 병합
result_df = pd.concat(dfs)

# index 초기화
result_df.reset_index(drop=True, inplace=True)

# csv 파일로 저장
result_df.to_csv("type3.csv", index=False, encoding='utf-8-sig')
result_df.to_excel('type3.xlsx', index=False)

# 출력
result_df.tail()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=55973): Max retries exceeded with url: /session/a114b7dc5128f28ac8f20d997b09c861/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f943218b760>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [266]:
bub21 = pd.read_excel('bub21.xlsx')[4:]
bub20 = pd.read_excel('bub20.xlsx')[4:]
bub19 = pd.read_excel('bub19.xlsx')[4:]
bub18 = pd.read_excel('bub18.xlsx')[4:]
bub17 = pd.read_excel('bub17.xlsx')[4:]
bub16 = pd.read_excel('bub16.xlsx')[4:]

bubin = pd.concat([bub21, bub20, bub19, bub18, bub17, bub16])

In [267]:
bubin

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
4,Audi - A4 35 TDI,71,4,5,3,0,0,0,0,0,0,0,0,83,법인,-,2021
5,Audi - A4 40 TDI quattro,11,2,1,1,0,0,0,0,0,0,0,0,15,법인,-,2021
6,Audi - A4 40 TFSI,52,30,15,18,0,0,0,0,0,0,0,0,115,법인,-,2021
7,Audi - A4 45 TFSI quattro,0,4,0,1,0,0,0,0,0,0,0,0,5,법인,-,2021
8,Audi - A5 Cabriolet 45 TFSI quattro,0,3,9,1,0,0,0,0,0,0,0,0,13,법인,-,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,Volvo - XC70 D5,6,1,4,2,6,8,4,9,0,0,0,0,40,법인,-,2016
482,Volvo - XC90 D5 AWD,0,0,0,0,0,3,48,44,44,25,39,34,237,법인,-,2016
483,Volvo - XC90 T6 AWD,0,0,0,4,0,11,0,11,12,25,10,11,84,법인,-,2016
484,Volvo - XC90 T8 AWD,0,0,0,0,0,1,1,4,5,11,8,3,33,법인,-,2016


In [268]:
bubin.to_excel('bubin.xlsx')

In [256]:
result_df.head(1)

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,...,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,BMW - 218d Gran Coupe,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
dd = pd.read_excel('bubin.xlsx').iloc[:,1:]

In [270]:
d21=pd.read_csv('result2021.csv')
d20=pd.read_csv('result2020.csv')
d19=pd.read_csv('result2019.csv')
d18=pd.read_csv('result2018.csv')
d17=pd.read_csv('result2017.csv')
d16=pd.read_csv('result2016.csv')

In [271]:
result_df = pd.concat([d21, d20, d19, d18, d17, d16, dd])
result_df.to_csv('result_df.csv', index=False, encoding='utf-8-sig')
result_df.to_excel('result_df.xlsx', index=False)

In [272]:
result_df

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year
0,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021
1,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021
2,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021
3,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021
4,Mercedes-Benz - E 250,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,Volvo - XC70 D5,6,1,4,2,6,8,4,9,0,0,0,0,40,법인,-,2016
2720,Volvo - XC90 D5 AWD,0,0,0,0,0,3,48,44,44,25,39,34,237,법인,-,2016
2721,Volvo - XC90 T6 AWD,0,0,0,4,0,11,0,11,12,25,10,11,84,법인,-,2016
2722,Volvo - XC90 T8 AWD,0,0,0,0,0,1,1,4,5,11,8,3,33,법인,-,2016


In [273]:
import re

In [274]:
 d = [column for column in result_df["구분"]]

In [275]:
n = re.compile('(\w+\s?[-]?\w+)\s-|[A-Z]{2}[a-z]{4}')

In [276]:
brand = re.findall('(\w+\s?[-]?\w+)\s-', str(d))

In [277]:
re.findall('-\s(\w+\s?\w+\s?\w+\s+\w+)', str(d))

['218d Gran Coupe',
 'Continental GT V8',
 'Cayenne Turbo Coupe',
 'Tiguan Allspace 2',
 'A4 40 TDI quattro',
 'A4 40 TFSI',
 'A5 Cabriolet 45 TFSI',
 'A5 Sportback 40 TDI',
 'A5 Sportback 40 TFSI',
 'A5 Sportback 45 TFSI',
 'A6 40 TDI',
 'A6 40 TDI quattro',
 'A6 45 TDI quattro',
 'A6 45 TFSI',
 'A6 45 TFSI quattro',
 'A6 50 TDI quattro',
 'A7 45 TDI quattro',
 'A7 55 TFSI quattro',
 'A8 L 50 TDI',
 'Q5 40 TDI quattro',
 'Q8 50 TDI quattro',
 'SQ5 TDI',
 '118d Urban',
 '218d Gran Coupe',
 '320 Touring',
 '320d Touring',
 '320d xDrive',
 '420 Coupe',
 '420d Coupe',
 '523d xDrive',
 '530 xDrive',
 '620d Gran Turismo',
 '620d xDrive Gran Turismo',
 '730Ld xDrive',
 '740Li xDrive',
 '840 xDrive Gran Coupe',
 'Active Tourer',
 'M235 xDrive Gran Coupe',
 'M340 xDrive Touring',
 'M440 xDrive Coupe',
 'M550 xDrive',
 'XT5 3',
 'Bolt EV',
 'Camaro SS',
 'Jeep Cherokee 2',
 'Jeep Cherokee 3',
 'Jeep Gladiator',
 'Jeep Grand Cherokee 3',
 'Jeep Renegade 2',
 'Jeep Wrangler Rubicon 2',
 'Jeep Wra

In [278]:
driver.quit()

In [279]:
data = result_df

In [280]:
import re

In [281]:
d = [column for column in data["구분"]]

In [282]:
brand = re.findall('(\w+\s?[-]?\w+)\s-', str(d))
data["brand"] = pd.DataFrame(brand)

In [283]:
data['model'] = data['구분'].replace(to_replace ='(\w+\s?[-]?\w+)\s-', value = '', regex = True)

In [284]:
pd.DataFrame(data)

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year,brand,model
0,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,BMW,218d Gran Coupe
1,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,BMW,520
2,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Bentley,Continental GT V8
3,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Honda,CR-V Hybrid
4,Mercedes-Benz - E 250,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Mercedes-Benz,E 250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,Volvo - XC70 D5,6,1,4,2,6,8,4,9,0,0,0,0,40,법인,-,2016,BMW,XC70 D5
2720,Volvo - XC90 D5 AWD,0,0,0,0,0,3,48,44,44,25,39,34,237,법인,-,2016,BMW,XC90 D5 AWD
2721,Volvo - XC90 T6 AWD,0,0,0,4,0,11,0,11,12,25,10,11,84,법인,-,2016,BMW,XC90 T6 AWD
2722,Volvo - XC90 T8 AWD,0,0,0,0,0,1,1,4,5,11,8,3,33,법인,-,2016,BMW,XC90 T8 AWD


In [285]:
data.columns = ["구분", "1월", "2월", "3월", "4월", "5월", "6월", "7월", "8월", "9월", "10월", "11월", "12월", "total", "sales_type", "age", "year", "brand","model"]
data.head()

,구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,total,sales_type,age,year,brand,model
0,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,BMW,218d Gran Coupe
1,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,BMW,520
2,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Bentley,Continental GT V8
3,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Honda,CR-V Hybrid
4,Mercedes-Benz - E 250,1,0,0,0,0,0,0,0,0,0,0,0,1,개인-남자,~19,2021,Mercedes-Benz,E 250


In [286]:
data = pd.melt(data, id_vars=['brand', 'model', 'sales_type', 'age', 'year'], 
               value_vars=['1월', '2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월'], var_name='month')
data.head()

,brand,model,sales_type,age,year,month,value
0,BMW,218d Gran Coupe,개인-남자,~19,2021,1월,0
1,BMW,520,개인-남자,~19,2021,1월,1
2,Bentley,Continental GT V8,개인-남자,~19,2021,1월,0
3,Honda,CR-V Hybrid,개인-남자,~19,2021,1월,0
4,Mercedes-Benz,E 250,개인-남자,~19,2021,1월,1


In [287]:
data = data[["brand", "model", "value", "sales_type", "age", "month", "year"]]

In [288]:
data.to_csv('specified_model.csv', index=False, encoding='utf-8-sig')
data.to_excel('specified_model.xlsx')